# PyBaMM "show and tell"

PyBaMM is an open-source package to simulate physics-based models for lithium-ion batteries using state-of-the-art techniques.  The package is built on Python, and provides a flexible platform for implementation and comparison of new models and numerical methods, and an easy interaction with other Python packages. PyBaMM includes a broad range of standard and cutting-edge models for lithium-ion batteries including electrochemical, thermal and degradation effects.

## Basic example

PyBaMM includes the standard physics-based model for lithium-ion batteries, such as the Doyle-Fuller-Newman (DFN) model, also known as the Newman model or the pseudo-two-dimensional (P2D) model. This model can be solved with a few lines of code:

In [ ]:
import pybamm

model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model)
sim.solve([0, 3600])
sim.plot()

## Compare models

PyBaMM also includes other standard models, such as the Single Particle Model (with and without electrolyte). Due to PyBaMM's structure, it is very easy to compare different models.

In [ ]:
models = [
    pybamm.lithium_ion.SPM(),
    pybamm.lithium_ion.SPMe(),
    pybamm.lithium_ion.DFN(),
]

sims = []
for model in models:
    sim = pybamm.Simulation(model)
    sim.solve([0, 3600])
    sims.append(sim)
    
pybamm.dynamic_plot(sims)

## Change parameter values

So far we have run the models with the default parameter sets, but PyBaMM includes various parameter sets for different chemistries. It also allows users to define their own parameter sets. For example, to use the parameters for NMC 811 and graphite-SiO$_x$ from Chen et al. (2020), we simply need to add an additional line of code.

In [ ]:
model = pybamm.lithium_ion.DFN()
parameter_values = pybamm.ParameterValues("Chen2020")
parameter_values["Negative electrode thickness [m]"] = 100*1e-6
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])
sim.plot()

Following open-science principles, PyBaMM provides a list of the references used for the different components of the code (e.g. model, parameters, numerical solvers...). Adding `pybamm.print_citations()` at the end of the script provides a list with all the relevant citations.

In [ ]:
pybamm.print_citations()

For example, in this case we have Andersson et al. (2019) for the CasADI solver, Chen et al. (2020) for the parameter set, or Doyle et al. (1993) for the model.

We can also change individual parameter values

In [ ]:
model = pybamm.lithium_ion.DFN()
parameter_values["Negative electrode thickness [m]"] = 100e-6
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])
sim.plot()

Parameter values can also be modified "on the fly" at the call to `sim.solve` allowing the solve to be wrapped in an optimisation routine. 

## Run experiments

So far we have seen constant current discharges, but usually we want to simulate more complex cycles. This can be done using the `Experiment` class. The various steps of the experiment are defined in words and PyBaMM automatically assembles the correct model.

In [ ]:
model = pybamm.lithium_ion.DFN()

experiment = pybamm.Experiment(
    [
        ("Discharge at C/5 for 5 hours or until 3.3 V",
        "Rest for 1 hour",
        "Charge at 1 A until 4.1 V",
        "Hold at 4.1 V until 50 mA",
        "Rest for 1 hour"),
    ] * 2
)


sim = pybamm.Simulation(model, experiment=experiment)
sim.solve()
sim.plot()

This feature also allows to easily simulate drive cycles from data

In [ ]:
import pandas as pd

model = pybamm.lithium_ion.SPMe()

# import drive cycle from file
US06 = pd.read_csv("US06.csv", comment="#", header=None).to_numpy()

experiment = pybamm.Experiment(
    [
        "Run US06 (A) for 15 minutes",
        "Rest for 5 minutes",
        "Run US06 (A) for 10 minutes",
    ],
    period = "1 second",
    drive_cycles={"US06": US06}
)


sim = pybamm.Simulation(model, experiment=experiment)
sim.solve()
sim.plot()